In [15]:
!pip install transformers sentencepiece nltk pandas


In [16]:
import pandas as pd
import nltk
from transformers import pipeline
nltk.download('punkt_tab', quiet=True)
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [17]:
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model="nlptown/bert-base-multilingual-uncased-sentiment"
)


Device set to use cuda:0


In [18]:
def split_english_sentences(text):
    if pd.isna(text) or not str(text).strip():
        return []
    return nltk.sent_tokenize(str(text))
def split_arabic_sentences(text):
    if pd.isna(text) or not str(text).strip():
        return []
    return [s.strip() for s in str(text).split('.') if s.strip()]

In [19]:
def map_stars_to_sentiment(label):
    stars = int(label[0])
    if stars <= 2:
        return "negative"
    elif stars == 3:
        return "neutral"
    else:
        return "positive"

In [20]:
df=pd.read_csv("/content/care_opinions_website2.0.csv")

In [21]:
records = []
for idx, row in df.iterrows():
    en_sentences = split_english_sentences(row['Opinions'])
    for sent in en_sentences:
        result = sentiment_pipeline(sent[:512])[0]
        records.append({
            "review_id": idx,
            "language": "en",
            "sentence": sent,
            "sentiment": map_stars_to_sentiment(result['label'])
        })
    ar_sentences = split_arabic_sentences(row['Opinions_ar'])
    for sent in ar_sentences:
        result = sentiment_pipeline(sent[:512])[0]
        records.append({
            "review_id": idx,
            "language": "ar",
            "sentence": sent,
            "sentiment": map_stars_to_sentiment(result['label'])
        })

In [22]:
sentiment_df = pd.DataFrame(records)
sentiment_df



,review_id,language,sentence,sentiment
0,0,en,"My leaky aortic valve, was leading to worrying...",negative
1,0,en,"Fortunately, this led to me meeting Consultant...",positive
2,0,en,"He, without a doubt, was the star in my story.",positive
3,0,en,"However, during the event it became apparent t...",neutral
4,0,en,"I was going to require Open Heart Surgery, and...",negative
...,...,...,...,...
11036,239,ar,كانوا دائمًا يُطلعوننا على آخر المستجدات بشأن ...,positive
11037,239,ar,كان الطاقم يسألنا باستمرار عما إذا كانت لدينا ...,negative
11038,239,ar,لقد أصبحوا بمثابة عائلتنا الثانية,positive
11039,239,ar,لا يسعنا إلا أن نتقدم بجزيل الشكر والامتنان له...,negative


In [28]:
sentiment_pipeline('My leaky aortic valve, was leading to worrying')

[{'label': '2 stars', 'score': 0.41547441482543945}]